# Unify rasters to a base raster

In [11]:
import rasterio
import os
from importlib.resources import files
from rasterio.enums import Resampling
from pathlib import Path
from tqdm import tqdm

from beak.utilities.io import save_raster, create_file_folder_list, create_file_list, check_path
from beak.utilities.raster_processing import unify_raster_grids

In [12]:
BASE_PATH = files("beak.data")
BASE_NAME = "LAWLEY22"
BASE_SPATIAL = "EPSG_32615_RES_50_0"

input_folder = BASE_PATH / BASE_NAME / "EXPORT" / "EPSG_4326_RES_0_015" / "CLIPPED_USC" / "NUMERICAL_IMPUTED_SCALED_STANDARD"
output_folder = BASE_PATH / BASE_NAME / "EXPORT" / BASE_SPATIAL / "NUMERICAL_IMPUTED_SCALED_STANDARD"
base_raster = BASE_PATH / "BASE_RASTERS" / str(BASE_SPATIAL + ".tif")

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")
print(f"Base raster: {base_raster}")


Input folder: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD
Output folder: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_32615_RES_50_0\NUMERICAL_IMPUTED_SCALED_STANDARD
Base raster: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\BASE_RASTERS\EPSG_32615_RES_50_0.tif


In [13]:
# Create file and folder list
folders, _ = create_file_folder_list(input_folder)
folders.insert(0, input_folder)

file_list = []
for folder in folders:
  folder_files = create_file_list(folder, recursive=False)
  file_list.extend(folder_files)
  
print(f"Found {len(file_list)} files in {len(folders)} folders:")

for file in file_list:
  print(file)

Found 45 files in 1 folders:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Crust1_CrustalThickness.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Crust1_SedimentThickness.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Geology_BlackShale_Proximity.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Geology_CoverThickness.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_4326_RES_0_015\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Geology_Fault_

In [14]:
import multiprocessing as mp
from beak.utilities.raster_processing import unify_raster_grids

# Unify raster grids
results = unify_raster_grids(base_raster, file_list, resampling_method=Resampling.bilinear, same_extent=True, same_shape=True, n_workers=mp.cpu_count())

Starting parallel processing...
Done!


In [16]:
dry_run = False
if dry_run:
  print("Dry run, no files will be written.\n")
  
for i, file in enumerate(tqdm(file_list, total=len(file_list))):
  unified_raster = results[i][0]
  meta = results[i][1]
  
  out_path = output_folder / file.name
  check_path(Path(os.path.dirname(out_path)))
  
  if not dry_run:
    save_raster(out_path, array=unified_raster, dtype="float32", metadata=meta)


100%|██████████| 45/45 [00:30<00:00,  1.45it/s]
